In [ ]:
# -------------------------------------------
# Benin EDA Notebook
# -------------------------------------------

# 1️⃣ Setup imports and repo path
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from windrose import WindroseAxes

# Add repo root to sys.path so 'src' can be imported
repo_root = Path(r"D:\Python\Week_01\Assignment\solar-challenge-week0")
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

# Import src modules
from src.data_loader import load_country_data
from src.preprocess import preprocess_dataset
from src.analyze import run_full_analysis

# 2️⃣ Dataset configuration
FILENAME = "benin-malanville.csv"
COUNTRY_NAME = "benin"
TIMESTAMP_COL = "timestamp"

# 3️⃣ Load raw dataset
df_raw = load_country_data(FILENAME)
print(f"Raw dataset shape: {df_raw.shape}")
display(df_raw.head())

# -------------------------------------------
# 1️⃣ Summary Statistics & Missing-Value Report
# -------------------------------------------
display(df_raw.describe())
print(df_raw.isna().sum())
null_threshold = 0.05
cols_with_nulls = df_raw.isna().mean()[df_raw.isna().mean() > null_threshold].index.tolist()
print("Columns with >5% nulls:", cols_with_nulls)

# -------------------------------------------
# 2️⃣ Outlier Detection & Basic Cleaning
# -------------------------------------------
numeric_cols = ["ghi", "dni", "dhi", "moda", "modb", "ws", "wsgust"]
df_clean = preprocess_dataset(df_raw, country=COUNTRY_NAME)
df_zscores = df_clean[numeric_cols].apply(zscore)
outliers = (df_zscores.abs() > 3).sum()
print("Number of outliers per column:\n", outliers)

# Save cleaned dataset
df_clean.to_csv(f"data/{COUNTRY_NAME}_clean.csv", index=False)

# -------------------------------------------
# 3️⃣ Time Series Analysis
# -------------------------------------------
df_clean.plot(x=TIMESTAMP_COL, y=["ghi","dni","dhi","tamb"], figsize=(12,6))
plt.title("Time Series of Solar Radiation and Temperature")
plt.show()

# -------------------------------------------
# 4️⃣ Cleaning Impact
# -------------------------------------------
df_clean.groupby("cleaning")[["moda","modb"]].mean().plot(kind="bar")
plt.title("Effect of Cleaning on Module Measurements")
plt.show()

# -------------------------------------------
# 5️⃣ Correlation & Relationship Analysis
# -------------------------------------------
sns.heatmap(df_clean[["ghi","dni","dhi","tmoda","tmodb"]].corr(), annot=True, cmap="coolwarm")
plt.title("Correlation Matrix")
plt.show()

sns.scatterplot(x="ws", y="ghi", data=df_clean)
sns.scatterplot(x="wsgust", y="ghi", data=df_clean)
sns.scatterplot(x="wd", y="ghi", data=df_clean)
sns.scatterplot(x="rh", y="tamb", data=df_clean)
sns.scatterplot(x="rh", y="ghi", data=df_clean)

# -------------------------------------------
# 6️⃣ Wind & Distribution Analysis
# -------------------------------------------
ax = WindroseAxes.from_ax()
ax.bar(df_clean["wd"], df_clean["ws"], normed=True, opening=0.8, edgecolor='white')
ax.set_legend()
plt.show()

df_clean[["ghi","ws"]].hist(figsize=(10,4))
plt.show()

# -------------------------------------------
# 7️⃣ Temperature Analysis & Bubble Chart
# -------------------------------------------
plt.figure(figsize=(10,6))
plt.scatter(df_clean["ghi"], df_clean["tamb"], s=df_clean["rh"], alpha=0.5,
            c="orange", edgecolors="k")
plt.xlabel("GHI")
plt.ylabel("Tamb")
plt.title("GHI vs Tamb with RH Bubble Size")
plt.show()

sns.scatterplot(x="rh", y="tamb", data=df_clean)
sns.scatterplot(x="rh", y="ghi", data=df_clean)
